In [0]:
# 런타임 -> 런타임 유형변경 들어가서 gpu or tpu설정
# gpu버전 graphic 확인
!nvidia-smi

Sat Aug 24 08:33:56 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.40       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P0    27W /  70W |    205MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import validation_curve
from pprint import pprint
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier # rf분류기
from sklearn.decomposition import PCA
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import AdaBoostClassifier

In [0]:
# data load
train_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/bigcontest/preprocessing_data/train_final.csv')
test_data_1 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/bigcontest/preprocessing_data/test1_final.csv')
test_data_2 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/bigcontest/preprocessing_data/test2_final.csv')
survival_label = train_data['survival_time']
amount_spent_label = train_data['pledge_sum_spent']

In [0]:
print(train_data.shape)
print(test_data_1.shape)

(40000, 256)
(20000, 254)


In [0]:
train_data = train_data.drop('survival_time', axis=1)
train_data = train_data.drop('pledge_sum_spent', axis=1)
#train_data = train_data.drop('amount_binary', axis=1) # 추가
train_data = train_data.drop('acc_id', axis=1)

test_data_1 = test_data_1.drop('acc_id', axis=1)
test_data_2 = test_data_2.drop('acc_id', axis=1)

In [0]:
print(train_data.shape)
print(test_data_1.shape)

(40000, 253)
(20000, 253)


In [0]:
# data preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
train_data = StandardScaler().fit_transform(train_data)
test_data_1 = StandardScaler().fit_transform(test_data_1)
test_data_2 = StandardScaler().fit_transform(test_data_2)

In [0]:
import numpy as np
from sklearn.model_selection import train_test_split

In [0]:
x_train, x_test, y_train, y_test = train_test_split(train_data, survival_label, test_size=0.33, random_state=321)

In [0]:
est_AB = XGBClassifier()
score_AB = est_AB.fit(x_train, y_train).score(x_test, y_test)

est_RF = RandomForestClassifier()
score_RF = est_RF.fit(x_train, y_train).score(x_test, y_test)

est_Ensemble = VotingClassifier(estimators=[('AB', est_AB), ('RF', est_RF)], voting='soft', weights=[1,1])
score_ensemble = est_Ensemble.fit(x_train, y_train).score(x_test, y_test)

In [0]:
score_ensemble

0.6328787878787879

In [0]:
max(est_AB.feature_importances_)

0.033171285

In [0]:
from sklearn.feature_selection import SelectFromModel
model = SelectFromModel(est_AB, prefit=True)
X_train_new = model.transform(train_data)
X_test_new1 = model.transform(test_data_1)
X_test_new2 = model.transform(test_data_2)

In [0]:
print(X_train_new.shape)
print(X_test_new1.shape)

(40000, 79)
(20000, 79)


In [0]:
x_train, x_test, y_train, y_test = train_test_split(X_train_new, survival_label, test_size=0.33, random_state=321)

In [0]:
est_AB = XGBClassifier()
score_AB = est_AB.fit(x_train, y_train).score(x_test, y_test)

est_RF = RandomForestClassifier()
score_RF = est_RF.fit(x_train, y_train).score(x_test, y_test)

est_Ensemble = VotingClassifier(estimators=[('AB', est_AB), ('RF', est_RF)], voting='soft', weights=[1,1])
score_ensemble = est_Ensemble.fit(x_train, y_train).score(x_test, y_test)

In [0]:
score_ensemble

0.636969696969697

In [0]:
from sklearn.metrics import accuracy_score
predictions_1 = est_Ensemble.predict(X_test_new1)
predictions_2 = est_Ensemble.predict(X_test_new2)

df_1 = pd.DataFrame(predictions_1)
df_2 = pd.DataFrame(predictions_2)

#df_1 = df_1.rename(columns = {'0':'survival_time'})
#df_2 = df_2.rename(columns = {'0':'survival_time'})

df_1.to_csv('/content/drive/My Drive/Colab Notebooks/bigcontest/190824_test1_result1_4.csv')
df_2.to_csv('/content/drive/My Drive/Colab Notebooks/bigcontest/190824_test2_result1_4.csv')

In [0]:
df_2

,0
0,64
1,64
2,64
3,64
4,64
5,64
6,64
7,64
8,64
9,64


In [0]:
#https://scikit-learn.org/stable/modules/feature_selection.html